Version de SageMath utilisée : 9.3

# Trou Noir Chargé Statique en Relativité Intriquée 

L'action de la relativité intriquée est donnée par (sous sa forme tenseur-scalaire) :

$S = \int \mathrm{d}^4 x\sqrt{-g} \left( \frac{\varphi^2 R}{2\kappa} + \varphi \mathcal{L}_m \right)$

avec $g$ le determinant de la métrique, $R$ la courbure scalaire de la variété, $\kappa$ la constante d'Einstein et $\mathcal{L}_m$ le lagrangien de la matière. 

Les équations de champs sont les suivantes :

$G_{\mu\nu} = \frac{\kappa}{\varphi}T_{\mu\nu} + \frac{1}{\varphi^2} \left( \nabla_\mu \nabla_\nu - g_{\mu\nu} \Box \right) \varphi^2$

$\varphi = - \kappa \frac{\mathcal{L}_m}{R}$

En prenant la trace de la première équation, on trouve : $\frac{3}{\varphi^2}\Box \varphi^2 = R + \frac{\kappa}{\varphi} T = \frac{\kappa}{\varphi} \left(T - \mathcal{L}_m \right)$.

Dans le cas du trou noir chargé : $\mathcal{L}_m = -\frac{1}{2} F_{\mu\nu}F^{\mu\nu}$

### Définition de la variété, des paramètres et des coordonées

In [1]:
%display latex

In [2]:
M = Manifold(4, 'M', structure='Lorentzian')

In [3]:
m = var('m')
assume(m>=0)
q = var('q')

Le paramètre $m$ représente la masse du trou noir, $q$ sa charge.

In [4]:
u, v = var('u, v')

Les paramètres $u$ et $v$ sont reliés à $m$ et $q$ par les relations suivantes : $2m = u+\frac{11}{13}v$ et $q^2 = \frac{12}{13}uv$.

In [5]:
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:\varphi")

### La solution

In [6]:
"""La métrique"""

g = M.metric(name='g')
g[0,0] = -(1-u/r)*(1-v/r)**(15/13)
g[1,1] = 1/((1-u/r)*(1-v/r)**(7/13))
g[2,2] = r**2*(1-v/r)**(6/13)
g[3,3] = (r*sin(th))**2*(1-v/r)**(6/13)
g.display()

g = (u/r - 1)*(-v/r + 1)^(15/13) dt*dt - 1/((u/r - 1)*(-v/r + 1)^(7/13)) dr*dr + r^2*(-v/r + 1)^(6/13) dth*dth + r^2*(-v/r + 1)^(6/13)*sin(th)^2 dph*dph

On calcule le determinant, l'inverse et la connexion de Levi-Civita $\nabla$ de la métrique $g$ :

In [7]:
detg = g.determinant()
detg.display()

M --> R
(t, r, th, ph) |--> -(r^3*sin(th)^2 - r^2*v*sin(th)^2)*(r - v)^(7/13)*r^(6/13)

In [8]:
invg = g.inverse()
invg.display()

inv_g = -r^(28/13)/((r^2 - r*u - (r - u)*v)*(r - v)^(2/13)) d/dt*d/dt + (r - u)*(r - v)^(7/13)/r^(20/13) d/dr*d/dr + 1/((r - v)^(6/13)*r^(20/13)) d/dth*d/dth + 1/((r - v)^(6/13)*r^(20/13)*sin(th)^2) d/dph*d/dph

In [9]:
nab = g.connection(name=r'\nabla')

In [10]:
"""Le champ électromagnétique"""

A = M.one_form({X.frame(): [-sqrt(12*u*v/13)/r, 0, 0, 0]})

F = A.exterior_derivative()
F.display()

-2/13*sqrt(13)*sqrt(3)*sqrt(u)*sqrt(v)/r^2 dt/\dr

On calcule le Lagrangien $\mathcal{L}_m$ "on-shell" :

In [11]:
Lm = -F['_ij']*F.up(g)['^ij']/2
Lm.display()

M --> R
(t, r, th, ph) |--> 12/13*(r - v)^(5/13)*u*v/((r^4 - r^3*v)*r^(5/13))

In [12]:
"""Le champ phi"""

phi = M.scalar_field({X:(1-v/r)**(-2/13)}, name=r'\varphi')
phi.display()

\varphi: M --> R
   (t, r, th, ph) |--> (-v/r + 1)^(-2/13)

### Vérification de la solution

In [13]:
"""Equation de Maxwell"""

N = nab(phi*F.up(g))['_i^ij']
N.display()

0

On a bien l'équation de Maxwell qui est vérifiée : $\nabla_{\mu}\left(\varphi F^{\mu\nu}\right) = 0$.

In [14]:
"""Equation d'Einstein"""

Ric = g.ricci() #Le tenseur de Ricci
R = g.ricci_scalar() #La courbure scalaire
G = Ric-R*g/2 #Le tenseur d'Einstein

T = 2*(F.up(g,1)['_i^j']*F['_kj'] + g*Lm/2) #Le tenseur énergie impuslion

S = (nab(nab(phi**2)) - g*(phi**2).dalembertian())/(phi**2) #Le tenseur en plus dans l'équation

G == T/phi + S

True

On retrouve bien $G_{\mu\nu} = \frac{\kappa}{\varphi}T_{\mu\nu} + \frac{1}{\varphi^2} \left( \nabla_\mu \nabla_\nu - g_{\mu\nu} \Box \right) \varphi^2$. La constante $\kappa$ a été prise égale à $1$.

In [15]:
"""Equation du champ phi"""
(-Lm/R).display()

M --> R
(t, r, th, ph) |--> r^(2/13)/(r - v)^(2/13)

On retrouve la valeur du champ $\varphi$. On a bien l'équation : $\varphi = - \kappa \frac{\mathcal{L}_m}{R}$.

### Transformation conforme

En effectuant la transformation conforme $g_{\mu\nu} \longrightarrow h_{\mu\nu} = \varphi^2 g_{\mu\nu}$, l'action est changée : 

$S =\int \mathrm{d}^4 x  \sqrt{-h} \left( \frac{R_h}{2 \kappa} - \frac{3}{\kappa\varphi^2} h^{\mu\nu} \partial_{\mu}\varphi \partial_{\nu}\varphi  + \varphi \mathcal{L}_{mh} \right)$

où toutes les grandeurs avec $h$ en indice ont été calculées à partir de la nouvelle métrique.

La métrique solution des équations de champs est :

In [16]:
h = M.metric('h')
h[0,0] = -(1-u/r)*(1-v/r)**(11/13)
h[1,1] = 1/((1-u/r)*(1-v/r)**(11/13))
h[2,2] = r**2*(1-v/r)**(2/13)
h[3,3] = (r*sin(th))**2*(1-v/r)**(2/13)
h.display()

h = (u/r - 1)*(-v/r + 1)^(11/13) dt*dt - 1/((u/r - 1)*(-v/r + 1)^(11/13)) dr*dr + r^2*(-v/r + 1)^(2/13) dth*dth + r^2*(-v/r + 1)^(2/13)*sin(th)^2 dph*dph

In [17]:
deth = h.determinant()
deth.display()

M --> R
(t, r, th, ph) |--> -(r - v)^(4/13)*r^(48/13)*sin(th)^2

In [18]:
nabh = h.connection()

In [19]:
"""Equation de Maxwell"""

nabh(phi*F.up(h))['_i^{ji}'].display()

0

In [20]:
"""Equation du dilaton"""

(nabh((ln(phi)).derivative().up(h))['_i^i']-1/12*phi*F['_{ij}']*F.up(h)['^{ij}']).display()

M --> R
(t, r, th, ph) |--> 0

In [22]:
"""Invariance conforme de l'action de l'électromagnétisme"""

Lmh = -F['_ij']*F.up(h)['^ij']/2

sqrt(-detg)*Lm == sqrt(-deth)*Lmh

True

In [23]:
Th = 2*(F.up(h,1)['_i^j']*F['_kj'] + h*Lmh/2)
Th.display()

12/13*(r*u - u^2)*(r - v)^(11/13)*v/r^(76/13) dt*dt - 12/13*u*v/((r^3 - r^2*u)*(r - v)^(11/13)*r^(2/13)) dr*dr + 12/13*(r - v)^(2/13)*u*v/r^(28/13) dth*dth + 12/13*(r - v)^(2/13)*u*v*sin(th)^2/r^(28/13) dph*dph

In [24]:
"""Equation d'Einstein"""

Rich = h.ricci()
a = 1/sqrt(12)
P = -1/(2*a)*ln(phi)
Sh = 2*P.derivative()*P.derivative()

(Rich - phi*Th - Sh).display()

0

On retrouve bien $\sqrt{-g} \mathcal{L}_m = \sqrt{-h} \mathcal{L}_{mh}$.

In [25]:
"""La nouvelle courbure scalaire"""

Rh = h.ricci_scalar()
Rh.display()

r(h): M --> R
   (t, r, th, ph) |--> 24/169*(r - u)*v^2/((r^4 - r^3*v)*(r - v)^(2/13)*r^(11/13))

In [26]:
"""Transformation de la courbure scalaire"""

f = ln(phi)

df = f.derivative()

(R-6*f.dalembertian() - 6*invg(df,df)).display()

M --> R
(t, r, th, ph) |--> 24/169*(r - u)*v^2/((r^4 - r^3*v)*(r - v)^(6/13)*r^(7/13))

In [27]:
(phi**2*Rh).display()

\varphi^2*r(h): M --> R
   (t, r, th, ph) |--> 24/169*(r - u)*v^2/((r^4 - r^3*v)*(r - v)^(6/13)*r^(7/13))

On retrouve la transformation de la courbure scalaire : $R - 6\Box f - 6 g^{\mu\nu}f_{\mu}f_{\nu} = \varphi^2 R_h$. (On a défini $f = \mathrm{ln}\varphi$ et $f_{\mu} = \partial_{\mu}f$)

### Tenseur de Weyl

In [28]:
C = (g.weyl()).down(g)
C[:]

[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
  [[0,
    -1/169*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)*(r - v)^(2/13)/((r^5 - r^4*v)*r^(2/13)),
    0,
    0],
   [1/169*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)*(r - v)^(2/13)/((r^5 - r^4*v)*r^(2/13)),
    0,
    0,
    0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[0,
    0,
    1/338*(169*r^3*u - 169*r^2*u^2 - 4*(35*r^2 - 109*r*u + 74*u^2)*v^2 + 13*(11*r^3 - 47*r^2*u + 36*r*u^2)*v)*(r - v)^(2/13)/r^(54/13),
    0],
   [0, 0, 0, 0],
   [-1/338*(169*r^3*u - 169*r^2*u^2 - 4*(35*r^2 - 109*r*u + 74*u^2)*v^2 + 13*(11*r^3 - 47*r^2*u + 36*r*u^2)*v)*(r - v)^(2/13)/r^(54/13),
    0,
    0,
    0],
   [0, 0, 0, 0]],
  [[0,
    0,
    0,
    1/338*(169*r^3*u*sin(th)^2 - 169*r^2*u^2*sin(th)^2 - 4*(35*r^2*sin(th)^2 - 109*r*u*sin(th)^2 + 74*u^2*sin(th)^2)*v^2 + 13*(11*r^3*sin(th)^2 - 47*r^2*u*sin(th)^2 + 36*r*u^2*sin(th)^2)*v)*(r - v)^(2/13)/r^(54/13)],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [-1/338*(169*r^3*u*sin(th)^2 - 169*r^2*u^2*sin(th)^2 - 4*(35*r^2*sin(th)^2 - 109*r*u*sin(th)^2 + 74*u^2*sin(th)^2)*v^2 + 13*(11*r^3*sin(th)^2 - 47*r^2*u*sin(th)^2 + 36*r*u^2*sin(th)^2)*v)*(r - v)^(2/13)/r^(54/13),
    0,
    0,
    0]]],
 [[[0,
    1/169*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)/((r - v)^(11/13)*r^(54/13)),
    0,
    0],
   [-1/169*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)/((r - v)^(11/13)*r^(54/13)),
    0,
    0,
    0],
   [0, 0, 0, 0],
   [0, 0, 0, 0]],
  [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
  [[0, 0, 0, 0],
   [0,
    0,
    -1/338*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)/((r^2 - r*u - (r - u)*v)*(r - v)^(7/13)*r^(6/13)),
    0],
   [0,
    1/338*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)/((r^2 - r*u - (r - u)*v)*(r - v)^(7/13)*r^(6/13)),
    0,
    0],
   [0, 0, 0, 0]],
  [[0, 0, 0, 0],
   [0,
    0,
    0,
    -1/338*(169*r^2*u*sin(th)^2 - 4*(35*r*sin(th)^2 - 74*u*sin(th)^2)*v^2 + 13*(11*r^2*sin(th)^2 - 36*r*u*sin(th)^2)*v)/((r^2 - r*u - (r - u)*v)*(r - v)^(7/13)*r^(6/13))],
   [0, 0, 0, 0],
   [0,
    1/338*(169*r^2*u*sin(th)^2 - 4*(35*r*sin(th)^2 - 74*u*sin(th)^2)*v^2 + 13*(11*r^2*sin(th)^2 - 36*r*u*sin(th)^2)*v)/((r^2 - r*u - (r - u)*v)*(r - v)^(7/13)*r^(6/13)),
    0,
    0]]],
 [[[0,
    0,
    -1/338*(169*r^3*u - 169*r^2*u^2 - 4*(35*r^2 - 109*r*u + 74*u^2)*v^2 + 13*(11*r^3 - 47*r^2*u + 36*r*u^2)*v)*(r - v)^(2/13)/r^(54/13),
    0],
   [0, 0, 0, 0],
   [1/338*(169*r^3*u - 169*r^2*u^2 - 4*(35*r^2 - 109*r*u + 74*u^2)*v^2 + 13*(11*r^3 - 47*r^2*u + 36*r*u^2)*v)*(r - v)^(2/13)/r^(54/13),
    0,
    0,
    0],
   [0, 0, 0, 0]],
  [[0, 0, 0, 0],
   [0,
    0,
    1/338*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)*(r - v)^(6/13)/((r^3 - r^2*u + (r - u)*v^2 - 2*(r^2 - r*u)*v)*r^(6/13)),
    0],
   [0,
    -1/338*(169*r^2*u - 4*(35*r - 74*u)*v^2 + 13*(11*r^2 - 36*r*u)*v)*(r - v)^(6/13)/((r^3 - r^2*u + (r - u)*v^2 - 2*(r^2 - r*u)*v)*r^(6/13)),
    0,
    0],
   [0, 0, 0, 0]],
  [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
  [[0, 0, 0, 0],
   [0, 0, 0, 0],
   [0,
    0,
    0,
    1/169*(169*r^2*u*sin(th)^2 - 4*(35*r*sin(th)^2 - 74*u*sin(th)^2)*v^2 + 13*(11*r^2*sin(th)^2 - 36*r*u*sin(th)^2)*v)/((r - v)^(7/13)*r^(6/13))],
   [0,
    0,
    -1/169*(169*r^2*u*sin(th)^2 - 4*(35*r*sin(th)^2 - 74*u*sin(th)^2)*v^2 + 13*(11*r^2*sin(th)^2 - 36*r*u*sin(th)^2)*v)/((r - v)^(7/13)*r^(6/13)),
    0]]],
 [[[0,
    0,
    0,
    -1/338*(169*r^3*u*sin(th)^2 - 169*r^2*u^2*sin(th)^2 - 4*(35*r^2*sin(th)^2 - 109*r*u*sin(th)^2 + 74*u^2*sin(th)^2)*v^2 + 13*(11*r^3*sin(th)^2 - 47*r^2*u*sin(th)^2 + 36*r*u^2*sin(th)^2)*v)*(r - v)^(2/13)/r^(54/13)],
   [0, 0, 0, 0],
   [0, 0, 0, 0],
   [1/338*(169*r^3*u*sin(th)^2 - 169*r^2*u^2*sin(th)^2 - 4*(35*r^2*sin(th)^2 - 109*r*u*sin(th)^2 + 74*u^2*sin(th)^2)*v^2 + 13*(11*r^3*sin(th)^2 - 47*r^2*u*sin(th)^2 + 36*r*u^2*sin(th)^2)*v)*(r - v)^(2/13)/r^(54/13),
    0,
    0,
    0]],
  [[0, 0, 0, 0],
   [0,
    0,
    0,

In [29]:
"""Définition d'une tétrade nulle"""

K = M.one_form({X.frame(): [sqrt(-g[0,0]/2), sqrt(g[1,1]/2), 0, 0]})
L = M.one_form({X.frame(): [sqrt(-g[0,0]/2), -sqrt(g[1,1]/2), 0, 0]})
Mb = M.one_form({X.frame(): [0, 0, sqrt(g[2,2]/2), i*sqrt(g[3,3]/2)]})
MB = M.one_form({X.frame(): [0, 0, sqrt(g[2,2]/2), -i*sqrt(g[3,3]/2)]})
k = K.up(g)
l = L.up(g)
m = Mb.up(g)
mb = MB.up(g)

In [30]:
"""Première composante indépendante du tenseur de Weyl"""

P00 = k*m*k*m
P0 = C['_{abcd}']*P00['^{abcd}']
P0.display()

zero: M --> R
   (t, r, th, ph) |--> 0

In [31]:
"""Deuxième composante indépendante du tenseur de Weyl"""

P11 = k*l*k*m
P1 = C['_{abcd}']*P11['^{abcd}']
P1.display()

zero: M --> R
   (t, r, th, ph) |--> 0

In [32]:
"""Trosième composante indépendante du tenseur de Weyl"""

P22 = k*m*mb*l
P2 = C['_{abcd}']*P22['^{abcd}']
P2.expr().factor()

1/676*(-1)^(5/13)*(169*r^2*u + 143*r^2*v - 468*r*u*v - 140*r*v^2 + 296*u*v^2)*((-1)^(9/13)*r^2 - (-1)^(9/13)*r*u - (-1)^(9/13)*r*v + (-1)^(9/13)*u*v - abs(-r + u)*abs(-r + v))/(r^(46/13)*(-r + v)^(19/13)*abs(-r + u)*abs(-r + v))

In [33]:
"""Quatrième composante indépendante du tenseur de Weyl"""

P33 = l*k*l*mb
P3 = C['_{abcd}']*P33['^{abcd}']
P3.display()

zero: M --> R
   (t, r, th, ph) |--> 0

In [34]:
"""Cinquième composante indépendante du tenseur de Weyl"""

P44 = l*mb*l*mb
P4 = C['_{abcd}']*P44['^{abcd}']
P4.display()

zero: M --> R
   (t, r, th, ph) |--> 0

Puisque seul $\Psi_2$ est non nul, l'espace-temps décrit ici est de type D.

### Solution avec monopôle magnétique

En changeant $\varphi$ en $\varphi_m=\frac{1}{\varphi}$ et $F_{\mu\nu}$ en $F_{m\mu\nu} = \varphi \frac{1}{2} \epsilon_{\mu\nu\lambda\rho} F^{\lambda\rho}$, on trouve une solution décrivant un monopôle magnétique.

In [35]:
"""Nouveau champ phi"""

phim = M.scalar_field(name=r'\varphi_m')
phim = 1/phi
phim.display()

1/\varphi: M --> R
   (t, r, th, ph) |--> (r - v)^(2/13)/r^(2/13)

La métrique $h$ n'a pas changé selon cette transformation. Mais pour retrouver la solution correspondant au trou noir chargé (magnétiquement), on effectue la transformation suivante : $g_{m\mu\nu} = \frac{1}{\varphi_m^2} h_{\mu\nu} = \varphi^2h_{\mu\nu}$.

In [36]:
"""Nouvelle métrique"""

gm = M.metric(name='g_m')
gm = h*phi**2
gm.display()

\varphi^2*h = -(r - u)*(r - v)^(7/13)/r^(20/13) dt*dt + r^(28/13)/((r^2 - r*u - (r - u)*v)*(r - v)^(2/13)) dr*dr + r^(28/13)/(r - v)^(2/13) dth*dth + r^(28/13)*sin(th)^2/(r - v)^(2/13) dph*dph

In [37]:
Rm = gm.ricci_scalar() #La courbure scalaire de la nouvelle métrique

In [38]:
Rm.expr().factor()

12/13*(r - v)^(2/13)*u*v/r^(54/13)

In [39]:
R.expr().factor()

-12/13*u*v/((r - v)^(6/13)*r^(46/13))

In [40]:
"""Définition du tenseur dualiseur de Levi-Civita"""

dt = M.one_form({X.frame(): [1, 0, 0, 0]})
dr = M.one_form({X.frame(): [0, 1, 0, 0]})
dth = M.one_form({X.frame(): [0, 0, 1, 0]})
dph = M.one_form({X.frame(): [0, 0, 0, 1]})

e = dt.wedge(dr.wedge(dth.wedge(dph))) #Le symbôle de Levi-Civita

E = e*sqrt(-detg) #Le tenseur de Levi-Civita

In [41]:
"""Le nouveau tenseur électromagnétique"""

Fm = M.tensor_field(0,2, name='F_m')
Fm = (1/2)*phi*E['_{abij}']*F.up(g)['^{ij}']
Fm.display()

2/13*sqrt(13)*sqrt(3)*sqrt(u)*sqrt(v)*sin(th) dth/\dph

In [42]:
"""Le nouveau lagrangien on-shell"""

Lmm = - Fm['_{ij}']*Fm.up(gm)['^{ij}']/2
Lmm.display()

M --> R
(t, r, th, ph) |--> -12/13*(r - v)^(4/13)*u*v/r^(56/13)

In [43]:
Lm.expr().factor()

12/13*u*v/((r - v)^(8/13)*r^(44/13))

In [44]:
(-Lmm/Rm).display()

M --> R
(t, r, th, ph) |--> (r - v)^(2/13)/r^(2/13)

On a toujours l'équation qui est vérifiée : $\varphi_m = -\kappa \frac{\mathcal{L}_{mh}}{R_m}$.

### Comparaison des solutions : chargé électriquement et chargé magnétiquement

#### La courbure scalaire

In [46]:
R.display()

r(g): M --> R
   (t, r, th, ph) |--> -12/13*u*v/((r - v)^(6/13)*r^(46/13))

In [47]:
Rm.display()

r(\varphi^2*h): M --> R
   (t, r, th, ph) |--> 12/13*(r*u*v - u*v^2)/((r - v)^(11/13)*r^(54/13))

#### Le tenseur de Weyl et ses composantes indépendantes

In [48]:
"""Le tenseur de Weyl et la tétrade nulle de la solution chargée magnétiquement"""

Cm = (gm.weyl()).down(gm)

Km = M.one_form({X.frame(): [sqrt(-gm[0,0]/2), sqrt(gm[1,1]/2), 0, 0]})
Lm = M.one_form({X.frame(): [sqrt(-gm[0,0]/2), -sqrt(gm[1,1]/2), 0, 0]})
Mbm = M.one_form({X.frame(): [0, 0, sqrt(gm[2,2]/2), i*sqrt(gm[3,3]/2)]})
MBm = M.one_form({X.frame(): [0, 0, sqrt(gm[2,2]/2), -i*sqrt(gm[3,3]/2)]})
km = Km.up(gm)
lm = Lm.up(gm)
mm = Mbm.up(gm)
mbm = MBm.up(gm)

In [49]:
"""Première composante indépendante"""

P00m = km*mm*km*mm
P0m = Cm['_{abcd}']*P00m['^{abcd}']
P0m.display()

zero: M --> R
   (t, r, th, ph) |--> 0

In [50]:
"""Deuxième composante indépendante"""

P11m = km*lm*km*mm
P1m = Cm['_{abcd}']*P11m['^{abcd}']
P1m.display()

zero: M --> R
   (t, r, th, ph) |--> 0

In [51]:
"""Trosième composante indépendante"""

P22m = km*mm*mbm*lm
P2m = Cm['_{abcd}']*P22m['^{abcd}']
P2m.display()

M --> R
(t, r, th, ph) |--> -1/676*(169*(-1)^(6/13)*r^4*u - 169*(-1)^(6/13)*r^3*u^2 + 4*(35*(-1)^(6/13)*r^2 - 109*(-1)^(6/13)*r*u + 74*(-1)^(6/13)*u^2)*v^3 - (283*(-1)^(6/13)*r^3 - 1047*(-1)^(6/13)*r^2*u + 764*(-1)^(6/13)*r*u^2)*v^2 + 13*(11*(-1)^(6/13)*r^4 - 60*(-1)^(6/13)*r^3*u + 49*(-1)^(6/13)*r^2*u^2)*v + (169*r^2*u*abs(-r + u) - 4*(35*r - 74*u)*v^2*abs(-r + u) + 13*(11*r^2 - 36*r*u)*v*abs(-r + u))*abs(-r + v))*(-r + v)^(2/13)/((r^5*abs(-r + u) - r^4*v*abs(-r + u))*r^(2/13)*abs(-r + v))

In [52]:
"""Quatrième composante indépendante"""

P33m = lm*km*lm*mbm
P3m = Cm['_{abcd}']*P33m['^{abcd}']
P3m.display()

zero: M --> R
   (t, r, th, ph) |--> 0

In [53]:
"""Cinquième composante indépendante"""

P44m = lm*mbm*lm*mbm
P4m = Cm['_{abcd}']*P44m['^{abcd}']
P4m.display()

zero: M --> R
   (t, r, th, ph) |--> 0

Seule la composante $\Psi_2$ ne s'annule pas.